In [2]:
import pandas as pd
import numpy as np


df = pd.read_csv(
    "shopify_orders.csv", parse_dates=['order_date']


)

In [3]:
print(f"Loaded {len(df)} orders")

Loaded 500 orders


In [7]:
df.head(3)

,order_id,customer_name,customer_email,product,category,quantity,unit_price,discount_pct,discount_amount,total,order_date,order_hour,payment_method,status,shipping_city,is_refunded,z_score_total
0,SHOP-00001,Richard Allen,lisa94@example.net,Hoodie Premium,Apparel,1,1200,10,120.0,1080.0,2025-05-21 11:14:00,11,GCash,Completed,North Robinmouth,False,-0.465772
1,SHOP-00002,Michael Martinez,daniel50@example.com,Hoodie Premium,Apparel,5,1200,0,0.0,6000.0,2025-01-17 08:05:00,8,GCash,Completed,Clarkchester,False,1.967932
2,SHOP-00003,Stephanie Mcgrath,xwhite@example.com,Classic T-Shirt,Apparel,5,450,0,0.0,2250.0,2025-11-29 19:34:00,19,COD,Completed,North Jade,False,0.112974


In [9]:
print(mean_total)
print(std_total)

2021.61
2021.61


In [10]:
df['z_score_total'] = (df['total'] - mean_total) / std_total

In [11]:
df.head(3)

,order_id,customer_name,customer_email,product,category,quantity,unit_price,discount_pct,discount_amount,total,order_date,order_hour,payment_method,status,shipping_city,is_refunded,z_score_total
0,SHOP-00001,Richard Allen,lisa94@example.net,Hoodie Premium,Apparel,1,1200,10,120.0,1080.0,2025-05-21 11:14:00,11,GCash,Completed,North Robinmouth,False,-0.465772
1,SHOP-00002,Michael Martinez,daniel50@example.com,Hoodie Premium,Apparel,5,1200,0,0.0,6000.0,2025-01-17 08:05:00,8,GCash,Completed,Clarkchester,False,1.967932
2,SHOP-00003,Stephanie Mcgrath,xwhite@example.com,Classic T-Shirt,Apparel,5,450,0,0.0,2250.0,2025-11-29 19:34:00,19,COD,Completed,North Jade,False,0.112974


In [12]:
df['flag_revenue_anomaly'] = np.abs(df['z_score_total']) > 2

In [15]:
print(f"\n Revenue Anomalies (Z > 2): {df['flag_revenue_anomaly'].sum()}")


 Revenue Anomalies (Z > 2): 5


In [16]:
print(df[df['flag_revenue_anomaly']][['order_id', 'product', 'total', 'z_score_total']])

       order_id            product    total  z_score_total
135  SHOP-00136    Canvas Tote Bag  56650.0      27.022220
163  SHOP-00164  Sticker Pack (10)  13800.0       5.826242
228  SHOP-00229         Coffee Mug  20800.0       9.288829
325  SHOP-00326         Coffee Mug  28800.0      13.246071
420  SHOP-00421    Canvas Tote Bag  74250.0      35.728152


In [19]:
q1 = np.percentile(df['quantity'],25)
q3= np.percentile(df['quantity'],75)
IQR = q3 - q1

lower_bound = q1 - 1.5 * IQR
upper_bound = q3 + 1.5 * IQR

In [20]:
df['flag_quantity_anomaly'] = (df['quantity'] < lower_bound) | (df['quantity'] > upper_bound)

In [21]:
df.head(3)

,order_id,customer_name,customer_email,product,category,quantity,unit_price,discount_pct,discount_amount,total,order_date,order_hour,payment_method,status,shipping_city,is_refunded,z_score_total,flag_revenue_anomaly,flag_quantity_anomaly
0,SHOP-00001,Richard Allen,lisa94@example.net,Hoodie Premium,Apparel,1,1200,10,120.0,1080.0,2025-05-21 11:14:00,11,GCash,Completed,North Robinmouth,False,-0.465772,False,False
1,SHOP-00002,Michael Martinez,daniel50@example.com,Hoodie Premium,Apparel,5,1200,0,0.0,6000.0,2025-01-17 08:05:00,8,GCash,Completed,Clarkchester,False,1.967932,False,False
2,SHOP-00003,Stephanie Mcgrath,xwhite@example.com,Classic T-Shirt,Apparel,5,450,0,0.0,2250.0,2025-11-29 19:34:00,19,COD,Completed,North Jade,False,0.112974,False,False


In [22]:
print(f"Number of Anomalies (IQR): {df['flag_quantity_anomaly'].sum()}")

Number of Anomalies (IQR): 5


In [23]:
print(f"Normal range:{lower_bound:0f} to {upper_bound:.0f}")

Normal range:-1.000000 to 7


In [24]:
DISCOUNT_THRESHOLD = 50
df['flag_discount_abuse'] = df['discount_pct'] > DISCOUNT_THRESHOLD

In [26]:
print(f"We see discount abuse (> {DISCOUNT_THRESHOLD}%) : {df['flag_discount_abuse'].sum}")





We see discount abuse (> 50%) : <bound method Series.sum of 0      False
1      False
2      False
3      False
4      False
       ...  
495    False
496     True
497    False
498    False
499    False
Name: flag_discount_abuse, Length: 500, dtype: bool>


In [29]:
refunded = df[df['is_refunded'] == True].copy()
refunded

,order_id,customer_name,customer_email,product,category,quantity,unit_price,discount_pct,discount_amount,total,order_date,order_hour,payment_method,status,shipping_city,is_refunded,z_score_total,flag_revenue_anomaly,flag_quantity_anomaly,flag_discount_abuse
10,SHOP-00011,Kenneth Rodriguez,jonathanshea@example.net,Phone Case,Accessories,5,350,10,175.0,1575.0,2025-10-10 00:40:00,10,COD,Refunded,Lake Christinachester,True,-0.220918,False,False,False
17,SHOP-00018,James Thornton,markbaker@example.com,Canvas Tote Bag,Bags,5,550,15,412.5,2337.5,2025-10-10 00:37:00,18,Credit Card,Refunded,Frankmouth,True,0.156257,False,False,False
115,SHOP-00116,Mr. Brian Kennedy DDS,scott31@example.com,Hoodie Premium,Apparel,2,1200,0,0.0,2400.0,2025-10-10 00:20:00,15,COD,Refunded,Priceborough,True,0.187173,False,False,False
179,SHOP-00180,Leslie Bishop,jknight@example.org,Hoodie Premium,Apparel,3,1200,90,3240.0,360.0,2025-10-10 01:09:00,9,PayPal,Refunded,North Edward,True,-0.821924,False,False,True
192,SHOP-00193,Christopher Mitchell,sharon66@example.net,Phone Case,Accessories,1,350,0,0.0,350.0,2025-10-10 02:46:00,2,COD,Refunded,Lake Isabellaborough,True,-0.826871,False,False,False
231,SHOP-00232,Mr. Marcus Hughes,whitakerdiane@example.org,Hoodie Premium,Apparel,2,1200,15,360.0,2040.0,2025-10-10 00:09:00,22,Credit Card,Refunded,South Charlesborough,True,0.009097,False,False,False
295,SHOP-00296,Gloria Bailey,robert81@example.org,Sticker Pack (10),Stationery,4,150,0,0.0,600.0,2025-10-10 01:23:00,19,COD,Refunded,East Connor,True,-0.703207,False,False,False
388,SHOP-00389,Robert Young,shawpamela@example.org,Hoodie Premium,Apparel,4,1200,10,480.0,4320.0,2025-10-10 01:27:00,10,PayPal,Refunded,Corymouth,True,1.136911,False,False,False
403,SHOP-00404,Seth Dominguez,pottervictor@example.org,Hoodie Premium,Apparel,5,1200,0,0.0,6000.0,2025-10-10 01:50:00,13,COD,Refunded,Lake Kyle,True,1.967932,False,False,False
409,SHOP-00410,Ernest Finley,hhood@example.org,Water Bottle,Home,5,600,10,300.0,2700.0,2025-10-10 01:48:00,22,GCash,Refunded,North Gerald,True,0.335569,False,False,False


In [30]:
refunded['refund_date'] = refunded['order_date'].dt.date

In [31]:
refunded

,order_id,customer_name,customer_email,product,category,quantity,unit_price,discount_pct,discount_amount,total,...,order_hour,payment_method,status,shipping_city,is_refunded,z_score_total,flag_revenue_anomaly,flag_quantity_anomaly,flag_discount_abuse,refund_date
10,SHOP-00011,Kenneth Rodriguez,jonathanshea@example.net,Phone Case,Accessories,5,350,10,175.0,1575.0,...,10,COD,Refunded,Lake Christinachester,True,-0.220918,False,False,False,2025-10-10
17,SHOP-00018,James Thornton,markbaker@example.com,Canvas Tote Bag,Bags,5,550,15,412.5,2337.5,...,18,Credit Card,Refunded,Frankmouth,True,0.156257,False,False,False,2025-10-10
115,SHOP-00116,Mr. Brian Kennedy DDS,scott31@example.com,Hoodie Premium,Apparel,2,1200,0,0.0,2400.0,...,15,COD,Refunded,Priceborough,True,0.187173,False,False,False,2025-10-10
179,SHOP-00180,Leslie Bishop,jknight@example.org,Hoodie Premium,Apparel,3,1200,90,3240.0,360.0,...,9,PayPal,Refunded,North Edward,True,-0.821924,False,False,True,2025-10-10
192,SHOP-00193,Christopher Mitchell,sharon66@example.net,Phone Case,Accessories,1,350,0,0.0,350.0,...,2,COD,Refunded,Lake Isabellaborough,True,-0.826871,False,False,False,2025-10-10
231,SHOP-00232,Mr. Marcus Hughes,whitakerdiane@example.org,Hoodie Premium,Apparel,2,1200,15,360.0,2040.0,...,22,Credit Card,Refunded,South Charlesborough,True,0.009097,False,False,False,2025-10-10
295,SHOP-00296,Gloria Bailey,robert81@example.org,Sticker Pack (10),Stationery,4,150,0,0.0,600.0,...,19,COD,Refunded,East Connor,True,-0.703207,False,False,False,2025-10-10
388,SHOP-00389,Robert Young,shawpamela@example.org,Hoodie Premium,Apparel,4,1200,10,480.0,4320.0,...,10,PayPal,Refunded,Corymouth,True,1.136911,False,False,False,2025-10-10
403,SHOP-00404,Seth Dominguez,pottervictor@example.org,Hoodie Premium,Apparel,5,1200,0,0.0,6000.0,...,13,COD,Refunded,Lake Kyle,True,1.967932,False,False,False,2025-10-10
409,SHOP-00410,Ernest Finley,hhood@example.org,Water Bottle,Home,5,600,10,300.0,2700.0,...,22,GCash,Refunded,North Gerald,True,0.335569,False,False,False,2025-10-10


In [36]:
refunded_counts = refunded.groupby('refund_date').size().reset_index(name='refund_count')
refunded_counts

,refund_date,refund_count
0,2025-10-10,12


In [38]:
suspicious_refund_days = refunded_counts[refunded_counts['refund_count'] > 3]
suspicious_refund_days

,refund_date,refund_count
0,2025-10-10,12


In [40]:
df['refund_date'] = df['order_date'].dt.date
df = df.merge(
    refunded_counts.rename(columns={'refund_count': 'daily_refund_count'}),
    on='refund_date',
    how='left'
)

In [41]:
df.head(1)

,order_id,customer_name,customer_email,product,category,quantity,unit_price,discount_pct,discount_amount,total,...,payment_method,status,shipping_city,is_refunded,z_score_total,flag_revenue_anomaly,flag_quantity_anomaly,flag_discount_abuse,refund_date,daily_refund_count
0,SHOP-00001,Richard Allen,lisa94@example.net,Hoodie Premium,Apparel,1,1200,10,120.0,1080.0,...,GCash,Completed,North Robinmouth,False,-0.465772,False,False,False,2025-05-21,NaN


In [43]:
df['daily_refund_count'] = df['daily_refund_count'].fillna(0).astype(int)

In [44]:
df.head(1)

,order_id,customer_name,customer_email,product,category,quantity,unit_price,discount_pct,discount_amount,total,...,payment_method,status,shipping_city,is_refunded,z_score_total,flag_revenue_anomaly,flag_quantity_anomaly,flag_discount_abuse,refund_date,daily_refund_count
0,SHOP-00001,Richard Allen,lisa94@example.net,Hoodie Premium,Apparel,1,1200,10,120.0,1080.0,...,GCash,Completed,North Robinmouth,False,-0.465772,False,False,False,2025-05-21,0


In [45]:
df['flag_refund_spike'] = df['daily_refund_count'] > 3

In [48]:
print(f'Refund Spike Days:{len(suspicious_refund_days)}')
print(f'Suspicious Refund Days:{len(suspicious_refund_days)}')

Refund Spike Days:1
Suspicious Refund Days:1


In [49]:
df['flag_late_night'] = df['order_hour'].between(0,4)

In [50]:
df.head(3)

,order_id,customer_name,customer_email,product,category,quantity,unit_price,discount_pct,discount_amount,total,...,shipping_city,is_refunded,z_score_total,flag_revenue_anomaly,flag_quantity_anomaly,flag_discount_abuse,refund_date,daily_refund_count,flag_refund_spike,flag_late_night
0,SHOP-00001,Richard Allen,lisa94@example.net,Hoodie Premium,Apparel,1,1200,10,120.0,1080.0,...,North Robinmouth,False,-0.465772,False,False,False,2025-05-21,0,False,False
1,SHOP-00002,Michael Martinez,daniel50@example.com,Hoodie Premium,Apparel,5,1200,0,0.0,6000.0,...,Clarkchester,False,1.967932,False,False,False,2025-01-17,0,False,False
2,SHOP-00003,Stephanie Mcgrath,xwhite@example.com,Classic T-Shirt,Apparel,5,450,0,0.0,2250.0,...,North Jade,False,0.112974,False,False,False,2025-11-29,0,False,False


In [ ]:
print(f'Late night Orders(12AM-5AM):{df['flag_late_night'].sum()}')

Late night Orders(12AM-5AM):10


In [54]:
email_counts = df.groupby('customer_email').size().reset_index(name='order_count')
suspicous_emails = email_counts[email_counts['order_count'] > 5]
print(suspicous_emails)

                     customer_email  order_count
419  suspicious.buyer@fakeemail.com            7
